# PCR report Work In Progress :)

In [ ]:
import pandas as pd
import os

from pcrep.parse_input import parse_analysis_filepath
from pcrep.constants import CONC_NAME, DIL_FINAL_FACTOR_NAME, DIL_TYPE_NAME, DIL_SAMPLE_DESCRIPTION_NAME, SAMPLE_ID_NAME, CV_COLNAME

INPUT_PCR_DATA = "./example/231108_GN004773-019/230811_GN004773-019_20230811_100101_999.csv"
CONFIG_DIR = "C:/work/pcr-report/data"
df = pd.read_csv(INPUT_PCR_DATA, delimiter=';', decimal=',')
# df = df.replace(',', '.', regex=True)
df[CONC_NAME] = df[CONC_NAME].astype('Float64')

parsedc = parse_analysis_filepath(INPUT_PCR_DATA)
ANALYSIS_DIR = parsedc['analysis_dir']

BASE_FILEPATH = os.path.join(
    ANALYSIS_DIR, '{}_{}'.format(parsedc['date'], parsedc['gn']))
display(ANALYSIS_DIR)
display(BASE_FILEPATH)

In [ ]:
df

In [ ]:
INPUT_CONCENTRATION_DATA = BASE_FILEPATH + '_conc.csv'
df_conc = pd.read_csv(INPUT_CONCENTRATION_DATA, sep=";", decimal=',')
CONC_NAME
df_conc.set_index([SAMPLE_ID_NAME], inplace=True)
df_conc

In [ ]:
from pcrep.constants import FDL_NAME, SAMPLE_NAME, SAMPLE_TYPE_NAME, SAMPLE_NUM_NAME


df.loc[:, [FDL_NAME]] = df[SAMPLE_NUM_NAME].map(
    df_conc[DIL_FINAL_FACTOR_NAME], na_action='ignore')


df.loc[:, [SAMPLE_NAME]] = df[SAMPLE_NUM_NAME].map(
    df_conc[DIL_SAMPLE_DESCRIPTION_NAME], na_action='ignore')


df.loc[:, [SAMPLE_TYPE_NAME]] = df[SAMPLE_NUM_NAME].map(
    df_conc[DIL_TYPE_NAME], na_action='ignore')


df = df.dropna(subset=[SAMPLE_TYPE_NAME])
df

In [ ]:
targets = df['Target'].unique()
display(targets)
samples = df['Sample description 1'].unique()
samples.sort()
display(samples)

## Compute results

In [ ]:
from pcrep.constants import WELL_RESULT_NAME
from pcrep.pcrep import result_fn


df.loc[:, [WELL_RESULT_NAME]] = df.apply(lambda x: result_fn(
    x['Conc(copies/µL)'], x['final dilution factor']), axis=1)

### Limits

In [ ]:
PLASMID_CONTROL_LIMITS_FILE = 'plasmid_control_limits.csv'
palsmid_control_limits = pd.read_csv(
    os.path.join(CONFIG_DIR, PLASMID_CONTROL_LIMITS_FILE))
palsmid_control_limits.set_index(['Target'], inplace=True)
palsmid_control_limits

In [ ]:
REFERENCE_CONTROL_LIMITS_FILE = 'reference_control_limits.csv'
reference_control_limits = pd.read_csv(
    os.path.join(CONFIG_DIR, REFERENCE_CONTROL_LIMITS_FILE))
reference_control_limits.set_index(['Target'], inplace=True)
reference_control_limits

How to access limits

In [ ]:
rcl = reference_control_limits
lmts = rcl.loc['IDT']
lmts['upper 3s action']

In [ ]:
method_limits = pd.read_csv('./data/method_limits.csv')
method_limits.set_index(['target_id'], inplace=True)
display(method_limits)

dc_limits = {'method': method_limits, 'reference_control': reference_control_limits,
             'plasmid_control': palsmid_control_limits}

In [ ]:
method_limits.loc['IDT']['Lower [vg/μl]']
mlmts = method_limits.loc['IDT']
mlmts['Lower [vg/μl]']

### Multiindex ????

In [ ]:
dfi = df.copy()
dfi.reset_index(inplace=True)
dfi.rename(columns={'Sample description 1': SAMPLE_ID_NAME}, inplace=True)
dfi.set_index([SAMPLE_ID_NAME, 'Target', 'Well'], inplace=True)
dfi.sort_index(inplace=True)
dfi.sort_index(axis=1)
dfi.drop(['Sample description 2', 'Sample description 3', 'Sample description 4',
          'TargetType', 'Supermix', 'Status', 'Experiment', 'SampleType'],
         axis=1, inplace=True)

Compute mean and standard deviation of `[vg/ml]`

In [ ]:
dfi.loc[:, ['mean [vg/ml]']
        ] = dfi.groupby(level=["sample_id", 'Target']).apply(lambda x: x['vg/ml'].mean())

dfi.loc[:, ['STDE']] = dfi.groupby(level=["sample_id", 'Target']).apply(
    lambda x: x['vg/ml'].std(ddof=0))
# dfi.head()

In [ ]:
from pcrep.check import cv_fn

dfi.loc[:, [CV_COLNAME]] = dfi.apply(lambda x: cv_fn(
    x['mean [vg/ml]'], x['STDE'], x['sample type']), axis=1)
dfi

### Method check

In [ ]:
from pcrep.check import method_check_fn
from pcrep.constants import VALUE_CHECK_NAME

dfi.loc[:, [VALUE_CHECK_NAME]] = dfi.apply(
    lambda x: method_check_fn(x, dc_limits), axis=1)


dfi.head()

### Droplets check

In [ ]:
from pcrep.check import droplets_check_fn
from pcrep.constants import DROPLET_CHECK_NAME

dfi.loc[:, [DROPLET_CHECK_NAME]] = dfi.apply(
    lambda x: droplets_check_fn(x), axis=1)
dfi.head()

#### Control check

In [ ]:
from pcrep.check import control_check_fn, warning_check_fn
from pcrep.constants import CONTROL_CHECK_NAME, WARNING_CHECK_NAME, CV_CHECK_NAME


dfi.loc[:, [CONTROL_CHECK_NAME]] = dfi.apply(
    lambda x: control_check_fn(x, dc_limits), axis=1)


dfi.loc[:, [WARNING_CHECK_NAME]] = dfi.apply(
    lambda x: warning_check_fn(x, dc_limits), axis=1)

dfi.head()

#### CV check

In [ ]:
from pcrep.check import cv_check

dfi.loc[:, [CV_CHECK_NAME]] = dfi.apply(
    lambda x: cv_check(x[CV_COLNAME]), axis=1)

### Analysis dataframe export to excel

In [ ]:
from pcrep.check import concat_comments

dfi = dfi.assign(comments=dfi.apply(lambda x: concat_comments(x), axis=1))
dfi
dfc = dfi.copy()

In [ ]:
dfc

In [ ]:
col_order = ['Sample', 'final dilution factor', 'Conc(copies/µL)',
             'vg/ml', 'mean [vg/ml]', 'STDE', 'CV [%]', 'comments',
             'Accepted Droplets', 'Positives', 'Negatives', 'sample type']
dfi = dfi.loc[:, col_order]
dfi

In [ ]:
# format_mapping = {'Conc(copies/µL)': '{:.2f}',
#                   'vg/ml': '{:.2e}',
#                   'mean [vg/ml]': '{:.2e}',
#                   'STDE': '{:.2e}',
#                   'CV [%]': '{:.2f}',
#                   'final dilution factor': '{:.0e}'
#                   }
# dff = dfi.style.format(format_mapping)
# display(dff)
# dff.to_excel(BASE_FILEPATH + '-data_analysis_raw.xlsx', engine='openpyxl')
# dfi.to_markdown(BASE_FILEPATH + '-data_analysis.md')

In [ ]:
from pcrep.xlswriter import analysis_to_excel

xls_analysis_file = BASE_FILEPATH + '-data_analysis.xlsx'
analysis_to_excel(dfi, xls_analysis_file)

#### Get sample...

In [ ]:
# def get_sample(df, samnple_num, target_id=None):
#     idx = pd.IndexSlice
#     if target_id:
#         return df.loc[idx[samnple_num, target_id, :], :]
#     else:
#         return df.loc[idx[samnple_num, :, :], :]
from pcrep.final import get_sample

tmps = get_sample(dfi, 2)
tmps

In [ ]:
idxs = pd.IndexSlice
tmps.loc[idxs[2, ['IDT'], :], :]['mean [vg/ml]'].values

In [ ]:
import json

params_file = './data/params.json'
with open(params_file) as json_file:
    check_params = json.load(json_file)
# print(check_params)

## Export report

### Create final (MS Word) dataframe

In [ ]:
tmps.index

In [ ]:
idxs = pd.IndexSlice
display(tmps.loc[idxs[2, ['IDT'], :], :]['mean [vg/ml]'].values[0])
display(tmps.loc[idxs[2, ['IDT'], :], :]['Sample'].values[0])
display(tmps.index[0])
tmps.index.get_level_values('Target').unique()

In [ ]:
tmps.index.get_level_values(SAMPLE_ID_NAME).unique()[0]
tmps['Sample'].array[0]

In [ ]:
from pcrep.final import make_final

dff = make_final(dfc, samples)


display(dff)

In [ ]:
from pcrep.xlswriter import final_to_excel

final_to_excel(dff, BASE_FILEPATH + '-final.xlsx')

#### Checks

In [ ]:
s = get_sample(dfc, 1)
s

### Markdown and word export

In [ ]:
# with open('md_intro.md', 'r') as f:
#     md_intro = f.read()
# with open('md_end.md', 'r') as f:
#     md_end = f.read()

# md_eval = final.to_markdown()

# md = md_intro + md_eval + md_end


# def save_md(file_path, md_txt):
#     try:
#         with open(file_path, 'w') as fl:
#             fl.write(md_txt)
#     except Exception as e:
#         print('Error: ' + str(e))


# MD_FILE = './example/230901_GN004308-086/230901_GN004308-086.md'
# save_md(MD_FILE, md)

# xls_path = os.path.splitext(MD_FILE)[0] + '.xlsx'
# final.to_excel(xls_path)

In [ ]:
# from pcrep import mdhandling

# with open(os.path.join(DATA_DIR, "config.json")) as json_file:
#     jd = json.load(json_file)
#     reference_doc = jd['reference_docx']
#     pdflatex_bin = jd['pdflatex_bin']
#     pandoc_bin = jd['pandoc_bin']

# mdhandling.md2docx(pandoc_bin, reference_doc, MD_FILE)
# print("Done.")

In [ ]:
# ! pip install jinja2
# ! pip install tabulate
# ! pip install xlsxwriter
# ! pip install mypy
# ! python -m pip install --upgrade pandas